In [1]:
import numpy as np
import mne
from mne.preprocessing import ICA, create_eog_epochs

def perform_ICA(streams, num):
    # Step 1: Load your EEG data from the streams
    if num == 1:
        data = streams[0].get_data()[:9, :]  # Get the first 9 channels
        sampling_rate = streams[0].info['sfreq']
    else:
        data = streams[1].get_data()[:9, :]  # Get the first 9 channels
        sampling_rate = streams[1].info['sfreq']

    # Step 2: Load your channel locations file
    chanlocs_path = r'G:\Users\Public\Documents\USB\Master\Segon semestre\Human_Machine_Interface\eeglab_2023\eeglab2023\sample_locs\Standard-10-20-Cap10.txt'
    montage = mne.channels.read_montage('standard_1020', ch_names=[f'EEG {i+1}' for i in range(9)])
    
    # Step 3: Create an MNE Raw object
    info = mne.create_info(ch_names=[f'EEG {i+1}' for i in range(data.shape[0])], sfreq=sampling_rate, ch_types='eeg')
    raw = mne.io.RawArray(data, info)
    raw.set_montage(montage)

    # Step 4: Preprocess the data (bandpass filter between 1-50 Hz)
    raw.filter(l_freq=1, h_freq=50)

    # Step 5: Run ICA
    ica = ICA(n_components=min(9, data.shape[0]), random_state=42)
    ica.fit(raw)

    return ica  # Return the ICA object for further use


In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
from mne import read_raw_xdf  # Requires MNE-Python for loading XDF files
from your_module import perform_ICA, Filtering_mu_beta, Generic_INFO, Filtering, EEG_Plot_Filtering, Topoplot, Topo_comparison  # Replace 'your_module' with your actual module name

# Load XDF files
xdf_file_path_right = r"G:\Users\Public\Documents\USB\Feina UPC\EEG\Experiments\PSYCHOPY+BITBRAIN\LAB_RECORDER\OFFLINE\DATASET\SUB_2\RIGHT\sub-P001\ses-S010\eeg\sub-P001_ses-S010_task-Default_run-001_eeg.xdf"
xdf_file_path_left = r"G:\Users\Public\Documents\USB\Feina UPC\EEG\Experiments\PSYCHOPY+BITBRAIN\LAB_RECORDER\OFFLINE\DATASET\SUB_2\LEFT\sub-P001\ses-S004\eeg\sub-P001_ses-S004_task-Default_run-001_eeg.xdf"

# Load data
streams_R, fileheader_R = read_raw_xdf(xdf_file_path_right, preload=True)
streams_L, fileheader_L = read_raw_xdf(xdf_file_path_left, preload=True)


# Determine sampling rate
try:
    sampling_rate = streams_R.info['sfreq']
except KeyError:
    sampling_rate = streams_R.info['sfreq']

# Perform ICA Filtering
try:
    EEG_DATA_R = perform_ICA(streams_R, 1)
except Exception:
    EEG_DATA_R = perform_ICA(streams_R, 2)

try:
    EEG_DATA_L = perform_ICA(streams_L, 1)
except Exception:
    EEG_DATA_L = perform_ICA(streams_L, 2)


# PWD & Histogram
Generic_INFO(EEG_DATA_R, "R")
Generic_INFO(EEG_DATA_L, "L")


# Filtering
eeg_data_mu_R, eeg_data_beta_R = Filtering_mu_beta(EEG_DATA_R)
eeg_data_mu_L, eeg_data_beta_L = Filtering_mu_beta(EEG_DATA_L)

# Plot filtered data for a specific channel (e.g., channel 5)
plt.figure(figsize=(10, 8))

plt.subplot(2, 1, 1)
plt.plot(eeg_data_mu_R[4, :])  # Adjust index for zero-based
plt.title('Filtered EEG Data (Mu Band) - Right')
plt.xlabel('Samples')
plt.ylabel('Amplitude')

plt.subplot(2, 1, 2)
plt.plot(eeg_data_beta_R[4, :])  # Adjust index for zero-based
plt.title('Filtered EEG Data (Beta Band) - Right')
plt.xlabel('Samples')
plt.ylabel('Amplitude')

plt.tight_layout()
plt.show()

# EEG data filtered (generic filtering)
eeg_filt_R = Filtering(EEG_DATA_R)
eeg_filt_L = Filtering(EEG_DATA_L)
eeg_filt_Rd = Filtering(EEG_DATA_Rd)

# Representation EEG_RIGHT
events_beta_R = EEG_Plot_Filtering(streams_R, eeg_data_beta_R, 'R', "beta")
events_mu_R = EEG_Plot_Filtering(streams_R, eeg_data_mu_R, 'R', "mu")
events_R = EEG_Plot_Filtering(streams_R, eeg_filt_R, 'R', "all")

# Representation EEG_LEFT
events_beta_L = EEG_Plot_Filtering(streams_L, eeg_data_beta_L, 'L', "beta")
events_mu_L = EEG_Plot_Filtering(streams_L, eeg_data_mu_L, 'L', "mu")
events_L = EEG_Plot_Filtering(streams_L, eeg_filt_L, 'L', "all")



# TOPOPLOT
mean_L_data = Topoplot(eeg_filt_L, events_L, sampling_rate, "L", "all bands")
mean_R_data = Topoplot(eeg_filt_R, events_R, sampling_rate, "R", "all bands")

# Comparison
Topo_comparison(mean_R_data, mean_L_data, "all bands")


ImportError: cannot import name 'read_raw_xdf' from 'mne' (C:\Users\Ferran\anaconda3\envs\TFM\Lib\site-packages\mne\__init__.py)